In [348]:
import math
import matplotlib.pyplot as plt

# create a function to calculate Lennard Jones interaction energy for an incremental range of distances, epsilon term and sigma term
'''def calculate_LJ(r_min, r_max,r_inc,o,e):
    """
    The Lennard Jones interation between two particles

    Parameters
    ----------
    r_min : integer
        The minimum r to calculate. The function will divide this value by 10.

    r_max : integer
        The maximum r to calculate. The function will divid this value by 10.

    r_inc : integer
        The incremental increase in r to run calculations. The function will divide this value by 10.

    r : float
        The distance between the two particles in reduced units

    o : float
        The distance at which the particle-particle potential enerty is zero.

    e : float
        The depth of the potential well

    Returns
    -------
    x_values : List
        List of distances used to run calculations
        
    y_values : List
        List of the resulting values of pairwise energy between two particles in reduced units
    """
    # create empty lists to store distances and pairwise energy values for plotting
    x_values = []
    y_values = []
    
    # create for loop to calculate pairwise energies over a range of distances
    for i in range(r_min,r_max,r_inc):
        r = i/10
        r6_term = math.pow((o/r),6)
        r12_term = math.pow(r6_term,2)
        pairwise_energy = 4*e*(r12_term - r6_term)
        
        # append values to lists for plotting
        x_values.append(r)
        y_values.append(pairwise_energy)
    return x_values, y_values'''

def calculate_LJ(r_ij):
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy


def calculate_distance(coord1, coord2):
    """
    Calculate the distance between two 3D coordinates.
   
    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates
    
    Returns
    -------
    distance: float
        The distance between the two points.
    """

    dx = coord2[0]-coord1[0]
    dy = coord2[1]-coord1[1]
    dz = coord2[2]-coord1[2]

    d = math.sqrt(math.pow(dx,2) + math.pow(dy,2) + math.pow(dz,2))
    
    return d

def calculate_total_energy(coordinates, cutoff):
    """
    Calculate the total Lennard Jones energy of a system of particles.

    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.
    cutoff : float
        The cut-off distance.

    Returns
    -------
    total_energy : float
        The total pairwise Lennard Jones energy of the system of particles.
    """

    total_energy = 0

    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):

            dist_ij = calculate_distance(
                coordinates[i], coordinates[j]
            )

            if dist_ij < cutoff:
                interaction_energy = calculate_LJ(dist_ij)
                total_energy += interaction_energy

    return total_energy

In [349]:
atom1 = [0,-math.pow(2,(1/6)),0]
atom2 = [0,0,0]
atom3 = [0,math.pow(2,(1/6)),0]

coordinates = [atom1,atom2,atom3]
print(coordinates)


[[0, -1.122462048309373, 0], [0, 0, 0], [0, 1.122462048309373, 0]]


In [350]:
total_energy = 0
num_atoms = len(coordinates)

for i in range(num_atoms):
    for j in range(i+1, num_atoms):
        particle_i = coordinates[i]
        particle_j = coordinates[j]

        dist_ij = calculate_distance(particle_i, particle_j)
        total_energy += calculate_LJ(dist_ij)

        
print(total_energy)

-2.031005859375


In [351]:
test_energy = calc_total_energy(coordinates)
print(test_energy)

-2.031005859375


In [352]:
assert math.isclose(test_energy, -2, rel_tol=0.05)

## Checking a larger system of particles

In [353]:
# Provided function

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length


In [354]:
config1_file = "../data/sample_config1.txt"

sample_coords, box_length = read_xyz(config1_file)


In [355]:
!pwd

/home/ctomlin/chem_280/group-repository-sodium/day3


In [356]:
len(sample_coords)

800

In [357]:
box_length

10.0

In [358]:
sample_energy = calculate_total_energy(sample_coords, 3)
print(sample_energy)

-3487.454232861954


In [359]:
'''assert sample_energy == -4351.5'''

'assert sample_energy == -4351.5'

## Discussion Questions

1. nCr where n=number of particles r=2 representing a particle-particle interaction. This leads to a combinatorial explosion of interactions to take into account.
2C2 = 1
3C2 = 3
4C2 = 6
5C2 = 10
10C2 = 45
100C2 = 4950
By modifying a cutoff of 3sigma, we notice a slight increase in our total energy from -3487.45 to -3582.23. This increase still does not achieve the target of -4351.5
This cutoff does work, but does not meaningfully get us closer to the number we are aiming for.

2. Using a cutoff does decrease the amount of calculations that will need to be made which saves time and resources. Since the number of calculations scales exponentially
with sample number, the compute time is drastically decreased. This does, however, limit the fidelity of the results since those calculations are skipped over. 
This is often rectified with a tail correction equation that estimates what those interactions might have added.

3. 
	1. The maximum distance (sigma) = 5*sqrt(3) = ~8.66
	The farthest coordinates away any particle can be from another in a box defined by periodic boundaries is (5,5,5) given that the other particle's location is defined 
	by (0,0,0).
	2. The actual distance of the example ((0,0,0),(0,0,8)) with a box_length of 10 would be 2sigma since the periodic boundary allows for the interaction from the physical 
	opposite side of the box. 

In [360]:
def calculate_tail_correction(box_length, n_particles, cutoff):
    '''
    Calculates the tail correction to the internal energy found through the Lennard Jones Equation.
    
    Parameters
    ----------
    box_length : float
        length of the sides of the box created for the simulation
    n_particles : int
        number of particles created for the simulation
    cutoff : float
        cut-off distance
       
    Returns
    -------
    U_lrc : float
        this is the number corresponding with the energy of the particles outside of the defined cut-off limit    
    '''

    volume_of_box = math.pow((box_length),3)
    rc3_term = math.pow((1/cutoff),3)
    rc9_term = math.pow((rc3_term),3)
    inside_term = (((1/3)*rc9_term)-rc3_term)
    U_lrc = (8*math.pi*math.pow((n_particles),2)/(3*volume_of_box))*inside_term
    return U_lrc

In [361]:
calculate_tail_correction(box_length, len(sample_coords), cutoff=3)

-198.4888837441566

In [362]:
assert math.isclose(calculate_tail_correction(box_length, len(sample_coords), cutoff=3), -198.49, rel_tol=0.05) 
assert math.isclose(calculate_tail_correction(box_length, len(sample_coords), cutoff=4), -83.769, rel_tol=0.05) 

# Reflection

## Approach
I started by writing out all the elements of the equation and the important details of the task focusing on items I hadn't seen before. Then I started piecing together the elements of the code that were necessary, starting with the easiest elements. Once I had everything written in parts I put them together in a haphazard function on paper to then translate it into my Jupyterlab Notebook. This made the process really seamless to me because there was little chance of putting the code together incorrectly. After that had all been settled I started running the code to troubleshoot any errors of which the main one was a misunderstanding of the equation; the solution was just to move a pair of parentheses.  

## Observations
Overall, the task was not too difficult and the appropriate result was achieved. This tail correction equation can be checked against the values given by the [NIST](https://www.nist.gov/mml/csd/chemical-informatics-group/lennard-jones-fluid-reference-calculations-cuboid-cell) with the combination of the `assert` and `math.isclose()` functions. I chose to use the two cut-off values of 3 and 4 to ensure the code works with the cutoff parameter being changed. All other data for parameters was acquired from the sample_config1.txt file. 